### 5flowers 파라미터 튜닝

In [1]:
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf

# 꽃 이름
CLASS_NAMES = ['daisy','dandelion','roses','sunflowers', 'tulips']
CLASS_NAMES

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

#이미지 전처리
def read_and_decode(filename, reshape_dims):
  img = tf.io.read_file(filename)

  # 3차원 uint8 텐서로 변환
  img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)

  # 0.0 ~ 1.0 정규화
  img = tf.image.convert_image_dtype(img, tf.float32)

  return tf.image.resize(img, reshape_dims)

In [2]:
BATCH_SIZE = 32

def decode_csv(csv_row):
  record_defaults = ["path", "flower"]
  filename, label_string = tf.io.decode_csv(csv_row, record_defaults)
  img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
  label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))

  return img, label

train_dataset = (tf.data.TextLineDataset("c:/workspace3/data/images/flowers/train_set.csv").map(decode_csv)).take(800).batch(BATCH_SIZE)
eval_dataset = (tf.data.TextLineDataset("c:/workspace3/data/images/flowers/eval_set.csv").map(decode_csv)).take(200).batch(BATCH_SIZE)

In [5]:
# ! pip install keras-tuner

In [7]:
#하이퍼파라미터 튜닝

import keras_tuner as kt

def build_model(hp):
  lrate = hp.Float('lrate', 1e-4, 1e-1, sampling='log')
  l2 = hp.Choice('l2', values=[1e-1, 1e-2, 1e-3, 1e-4])
  num_hidden = hp.Int('num_hidden', min_value=8, max_value=128, step=8)
  regularizer = tf.keras.regularizers.l2(l2)

  model = tf.keras.Sequential([
              tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
              tf.keras.layers.Dense(num_hidden,
                                    kernel_regularizer=regularizer,
                                    activation=tf.keras.activations.relu),

              tf.keras.layers.Dense(len(CLASS_NAMES),
                                    kernel_regularizer=regularizer,
                                    activation='softmax')
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])
  return model

#확률적인 방법으로 최적의 하이퍼파라미터를 계산하는 방식
#GridSearch : 모든 조합을 테스트하는 방식
tuner = kt.BayesianOptimization(
    build_model,
    objective=kt.Objective('val_accuracy', 'max'),
    max_trials=10)  

tuner.search(
    train_dataset, validation_data=eval_dataset,
    epochs=20,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

Trial 10 Complete [00h 00m 19s]
val_accuracy: 0.47999998927116394

Best val_accuracy So Far: 0.47999998927116394
Total elapsed time: 00h 06m 25s
INFO:tensorflow:Oracle triggered exit


In [ ]:
print(tuner.get_best_hyperparameters(2)[0].values)
print(tuner.get_best_hyperparameters(2)[1].values)

In [ ]:
batch_size = 16
lrate = 0.0001
l2 = 0
dropout_prob = 0.2
num_hidden = [32, 16]

# layer의 파라미터, 출력값에 대한 패널티 설정
# l1 : Manhattan distance
# l2 : Euclidean distance

#regularizer = tf.keras.regularizers.l1_l2(l1, l2)
regularizer = tf.keras.regularizers.l2(l2)

layers = [tf.keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))]

for hno, nodes in enumerate(num_hidden):
  layers.extend([
    tf.keras.layers.Dense(nodes, kernel_regularizer=regularizer),
    tf.keras.layers.BatchNormalization(),                          
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(rate=dropout_prob),                        
  ])

layers.append(
    tf.keras.layers.Dense(len(CLASS_NAMES),
                          kernel_regularizer=regularizer,
                          activation='softmax')
)

model = tf.keras.Sequential(layers)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, validation_data=eval_dataset, epochs=10)

In [ ]:
# 모델의 성능 그래프

def training_plot(metrics, history):
  f, ax = plt.subplots(1, len(metrics), figsize=(5*len(metrics), 5))

  for idx, metric in enumerate(metrics):
    ax[idx].plot(history.history[metric], ls='dashed')
    ax[idx].set_xlabel("Epochs")
    ax[idx].set_ylabel(metric)
    ax[idx].plot(history.history['val_' + metric]);
    ax[idx].legend([metric, 'val_' + metric])

In [ ]:
training_plot(['loss', 'accuracy'], history)